# 🧠 Week 2 Assignment – Deep Research Assistant with Pushover Notifications

# Author: Bharat Puri
# Objective: Combine OpenAI Agents SDK, WebSearchTool, and Pushover alerts into a deployable Gradio app.



In [ ]:
!pip install git+https://github.com/openai/openai-agents-python.git

In [ ]:
# -------------------------------------------------------------
# 1. Imports
# -------------------------------------------------------------
from agents import Agent, WebSearchTool, Runner, trace
from dotenv import load_dotenv
from IPython.display import Markdown, display
import gradio as gr
import requests
import asyncio
import os

In [ ]:

# -------------------------------------------------------------
# 2. Load environment variables
# -------------------------------------------------------------
load_dotenv(override=True)

In [ ]:
# -------------------------------------------------------------
# 3. Define Agent Instructions and Setup
# -------------------------------------------------------------
INSTRUCTIONS = (
    "You are a research assistant. Given a search term, search the web for that term and "
    "produce a concise summary of the results. The summary must be 2-3 paragraphs and under 300 words. "
    "Capture the key insights clearly, without fluff or filler. Output only the summary text."
)

# Create the Deep Research agent
search_agent = Agent(
    name="DeepResearcher",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    tools=[WebSearchTool()]
)

In [ ]:
# -------------------------------------------------------------
# 4. Define Pushover Notification Function
# -------------------------------------------------------------
def send_pushover_notification(title: str, message: str):
    """
    Sends a push notification using the Pushover API.
    """
    token = os.environ.get("PUSHOVER_TOKEN")
    user = os.environ.get("PUSHOVER_USER")
    if not token or not user:
        print("⚠️ Pushover credentials not found in .env")
        return "Pushover not configured"

    url = "https://api.pushover.net/1/messages.json"
    data = {
        "token": token,
        "user": user,
        "title": title,
        "message": message,
    }
    try:
        response = requests.post(url, data=data, timeout=10)
        if response.status_code == 200:
            print("✅ Pushover notification sent successfully.")
            return "Notification sent!"
        else:
            print("⚠️ Failed to send Pushover notification:", response.text)
            return f"Error: {response.text}"
    except Exception as e:
        print("Error sending Pushover:", e)
        return f"Exception: {e}"


In [ ]:
# -------------------------------------------------------------
# 5. Define Async Research Logic
# -------------------------------------------------------------
async def run_research(query: str) -> str:
    """
    Runs the Deep Research agent on a given query and returns a formatted summary.
    """
    if not query.strip():
        return "Please provide a valid search query."

    print(f"🔍 Running research for topic: {query}")
    with trace("Deep Research Session"):
        result = await Runner.run(search_agent, query)

    summary = result.final_output
    print("✅ Research completed.")

    # Run the Pushover notification in a separate thread (non-blocking)
    loop = asyncio.get_event_loop()
    loop.run_in_executor(None, send_pushover_notification, "Deep Research Completed", f"Topic: {query}")

    return summary

In [ ]:
# -------------------------------------------------------------
# 6. Gradio Interface
# -------------------------------------------------------------
async def gradio_interface(query):
    """
    Wrapper for Gradio async interface.
    """
    summary = await run_research(query)
    return summary


iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(
        label="Enter research topic",
        placeholder="e.g., AI Agent Frameworks 2025",
        lines=2,
    ),
    outputs=gr.Markdown(label="Research Summary"),
    title="🔎 Deep Research Assistant with Pushover",
    description=(
        "An AI Agent that searches the web and summarizes insights for you.\n\n"
        "💡 You'll also get a **Pushover notification** when your research is ready!"
    ),
)

In [ ]:
# -------------------------------------------------------------
# 7. Launch App
# -------------------------------------------------------------
iface.launch()